# Given that someone likes certain animes, which other animes this person might like?

Let's start by importing the needed datasets and modules



In [ ]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.cluster import OPTICS

In [ ]:
anime_original = pd.read_csv('../input/anime-recommendations-database/anime.csv')
rating_original = pd.read_csv('../input/anime-recommendations-database/rating.csv')

In [ ]:
anime = anime_original.copy()
rating = rating_original.copy()

# Machine Learning recomendation system
### For this task we will use a colaborative system, in which we'll make recomendations based on the rating data of similar users.
Note that there are values of -1 in the rating column. This means that the user watched the film but didn't rate it. We'll choose to drop these values in this aproach. In another approach we may choose to input some values as the -1 rating is a significant proportion of the rating column, almost 15%

In [ ]:
condition = rating.loc[rating.rating == -1,:].index
rating.drop(condition,inplace=True)
rating

In [ ]:
# Creating an array with the Id's of the rated animes
anime_id_list = np.unique(rating.anime_id.values)
anime_id_list

## Creation of the ratings matrix 

### We'll create a matrix such that the ij element of the matrix will be equal to the rating given by user "i" to the anime "j". Whenever the user didn't rate the anime the entry will be assigned to 0.

There is a simples way to accomplish this task. The .pivot( ) method does exactly that, but unfortunately our dataset is huge so we don't have enough memory to make the calculations.

To solve this problem, I came up with a function that leads to the same result but as a Sparse Matrix so we don't have to store all the values in memory!

In [ ]:
from scipy.sparse import csr_matrix
from pandas.api.types import CategoricalDtype

user_c = CategoricalDtype(sorted(rating.user_id.unique()), ordered=True)
anime_c = CategoricalDtype(sorted(rating.anime_id.unique()), ordered=True)

row = rating.user_id.astype(user_c).cat.codes
col = rating.anime_id.astype(anime_c).cat.codes
sparse_matrix = csr_matrix((rating["rating"], (row, col)), \
                           shape=(user_c.categories.size, anime_c.categories.size))

In [ ]:
### Importing the sparse matrix previously calculated
import scipy.sparse
X_sparse = scipy.sparse.load_npz("../input/user-matrix/sparse_matrix.npz")




### The matrix is sparse because there are a ton of animes and obviously, most users didn't watch most of them

In [ ]:
#Less than 1% of the entries belong to given ratings
sparcity = (X_sparse.nonzero()[0].shape[0])/(X_sparse.shape[0]*X_sparse.shape[1])
sparcity*100

## In this approach, we'll use matrix factorization in order to predict all the entries which weren't assigned by the users, then we'll group similar users to finally give predictions about a new user
I've written a Gradient Descent algorithm to return two matrices P and Q such that their product approximates our original sparse matrix only at the entries that the users have given their ratings (non zero entries). This way the algorithm will learn the intrinsic taste of a user and will (hopefully!) generalize to the animes that weren't rated by the users. 

In [ ]:
def Gradient_Descent(X):
    X_sparse = X/np.max(X)           # normalizing values
    print(X_sparse)
    n_factors = 100
    n_steps =  500                # optimized
    alpha = 0.01                   # optimized
    
    #initializing the vectors randomly:
    p = np.random.normal(0, .01, (X_sparse.shape[0], n_factors))
    q = np.random.normal(0, .01, (n_factors, X_sparse.shape[1]))      # changed so as to follow matrix multiplication rule
    
    for k in range(0,n_steps):
        for (i,j) in zip(X_sparse.nonzero()[0],X_sparse.nonzero()[1]):
            err = X_sparse[i,j] - np.dot(p[i, :],q[:, j])              # multiply whole row and column
            p[i, :] = p[i, :] + alpha*q[:, j]*err                      # update whole row and column
            q[:, j] = q[:, j] + alpha*p[i, :]*err                      # update whole row and column
    print(np.dot(p, q)) 
    p = p*np.sqrt(np.max(X))                         # matrix multiplication rule for normalized values
    q = q*np.sqrt(np.max(X))                         # matrix multiplication rule for normalized values
    return (p,q)   

In [ ]:
#p,q = Gradient_Descent(X_sparse)

In [ ]:
#Loading both of the arrays
import numpy as np
p = np.load("../input/user-matrix/P_100.npy")
q = np.load("../input/user-matrix/Q_100.npy")
u = np.dot(p,q)
u

In [ ]:
u.mean()

### Great! Now we have a matrix with all the entries filled . We'll treat both the assigned values and the user-given values the same way without any distinction.

There is a problem, though... 

Let's see what the values of a well-known anime are. In this case let's take FullMetal Alchemist.

In [ ]:
anime.loc[anime.name == 'Fullmetal Alchemist: Brotherhood']

In [ ]:
np.where(anime_id_list == 5114 )

In [ ]:
print('The mean is: ',u[:,3936].mean())
print('The standard deviation is: ',u[:,3936].std())

Some animes have means greater than 9 (with low standard deviation). That means that the algorithm will consistently choose these animes as top recomendations just because their overall scores are so high (as we will see when I show the full algorithm).

To solve this, I chose to reduce the values of the animes with mean greater than 9 and greater than 8 by using a weighted mean with those values and the mean of all the animes to give a chance for the lower rated ones to compete. Of course we need to reduce the ratings of the >9-mean animes more than the >8 animes.


In [ ]:
#function to reduce the ratings of the best rated anime
def squeezing_top_values(u):
  p1 = 9        # paremeter to reduce the greater than 9-mean animes
  p2 = 200      # paremeter to reduce the greater than 8 and lower than 9 mean animes
  index1 = np.where(u.mean(axis=0)>9)
  index2 = np.where((u.mean(axis=0)>8) & (u.mean(axis=0)<9))
  k = u.mean()
  for i in index1:
    u[:,i] = (u[:,i]*p1 + k)/(p1+1)
  for i in index2:
    u[:,i] = (u[:,i]*p2 + k)/(p2+1)
  return u


u = squeezing_top_values(u)    


In [ ]:
# Now we have:
print('The mean now is: ',u[:,3936].mean())
print('The standard deviation now is: ',u[:,3936].std())

#That's better!

### Let's assign the user input to a vector


In [ ]:
''' This function takes the names and the ratings of the animes given 
by the new user and transforms them to the correct form such that the algorithm
will understand
'''

def user_input_scores(array):
    a = []
    scores = []
    for i in range(0,len(array)):
        a.append(anime.loc[anime.name == array[i][0]].anime_id.values[0]) 
    a = np.array(a)
    
    for i in range(0,len(anime_id_list) - len(array)):
        scores.append(0)
    
    for i in range(0,len(a)):
        index = np.where(anime_id_list == a[i])[0][0]
        scores.insert(index,array[i][1])
    scores = np.array(scores)
    return scores


### I've written a few customized lists of well known animes to test if the given recomendations make sense.

In [ ]:
# shonen
y1 = [('Death Note',8),('Naruto',10),('Hunter x Hunter (2011)',9)]
# sports
y2 = [('Haikyuu!!',10),('Diamond no Ace',10)]

# Slice of Life
y3 = [('Toradora!',10),('Mob Psycho 100',10),('Suzumiya Haruhi no Yuuutsu',9)]

# Mecha
y4 = [('Code Geass: Hangyaku no Lelouch',10),('Neon Genesis Evangelion',10),('Guilty Crown',9)]

# Music
y5 = [('Shigatsu wa Kimi no Uso',9),('K-On!',10)]

#Kids
y6 = [('Pokemon',10),('Digimon Adventure',10)]

### Finally, below are all the functions needed to finish the recomendation process. The way that the recomendations will be given is as follows:

First, we'll use the KMeans algorithm to Cluster our users and see in which group our new user will be assigned to. To understand this in a high level you could think that the clustering will divide the users between the ones who like more shonen, the ones who like more sports animes and so on...

Then, we'll know the similar users with our new user.

 Based on that we'll take the mean of all the animes ratings of those viewers and the recomended shows will be the ones with the highest means, simple isn't it?




In [ ]:
#Function to determine the positions of the animes that the user has seen:

def position_seen_animes(y):
  scores = user_input_scores(y)
  position = np.where(scores!=0)[0]
  return position

# Function to create an array from the users matrix with only the animes the new user has seen

def user_scores_seen_animes(X,y):
  positions = position_seen_animes(y)
  return X[:,positions]

#Function to apply Kmeans algorithm to get similar viewers

def similar_users_position(X,y):
  
  n_clusters = 8 ### The best number after exhaustive tests

  X_transformed = user_scores_seen_animes(X,y)
  y_transformed = user_input_scores(y)[position_seen_animes(y)].reshape(1,-1)
  kmeans = KMeans(n_clusters = n_clusters).fit(X_transformed)
  group = kmeans.predict(y_transformed)
    
  (array_labels,n_labels) = np.unique(kmeans.labels_,return_counts = True) #test to see if the clustering was well done
  #print(n_labels,group) -- test to see if the clustering was well done
  similar_user_position = np.where(kmeans.labels_ == group)[0]
  return similar_user_position


#Function to take the mean scores of the similar users in order to rank the animes

def mean_scores(X,y):
  similar_users_scores = []
  
  index = similar_users_position(X,y) #similar users positions on the user matrix
  scores = np.zeros(len(anime_id_list)) #inicializing 

  for i in index:
    similar_users_scores.append(X[i])

  similar_users_scores = np.array(similar_users_scores)

  mean_scores = similar_users_scores.sum(axis = 0)/similar_users_scores.shape[0] # Getting the mean_scores

  return mean_scores

#Function to determine the positions of the best rated shows

def best_rated_shows(X,y):
  mean_marks = mean_scores(X,y)
  index = np.flip(anime_id_list[np.argsort(mean_marks)[-10:]]) ### Choose how many recommendations will be displayed
  return index

#Function to determine the names of the best rated shows

def names_best_rated_shows(X,y):
  index = best_rated_shows(X,y)
  print('The best recommendations for you are:\n')
  for i in index:
        print(anime.loc[anime.anime_id == i,['name']].values[0][0])
        print('')








## We're good to go to use our algorithm! Let's print the recommendations for different types of users!

In [ ]:
# Shonen
r1 = names_best_rated_shows(u,y1)

In [ ]:
# Sport
r2 = names_best_rated_shows(u,y2)

In [ ]:
# Slice of life
r3 = names_best_rated_shows(u,y3)

In [ ]:
# Mecha
r4 = names_best_rated_shows(u,y4) 

In [ ]:
# Music
r5 = names_best_rated_shows(u,y5)

In [ ]:
# Kids
r6 = names_best_rated_shows(u,y6)

### Considerations and Future Work

* The algorithm is quite sensitive to changes in the scores entered by the user and works best with few animes (1-3) and high scores (>8)

* The algorithm could improve if other factors are considered, such as the genre and the watched but not rated animes

### Thanks a lot for your time reading this Notebook! Feel free to message me!